# LAB 04.02 - Buiding Datasets

In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()
init.endpoint

'https://m5knaekxo6.execute-api.us-west-2.amazonaws.com/dev-v0001/rlxmooc'

In [2]:
from local.lib.rlxmoocapi import submit, session
session.LoginSequence(endpoint=init.endpoint, session_id='2025.2', course_id=init.course_id, lab_id="L04.02", varname="student");

logging in as santiago.herrera6@udea.edu.co... please wait
you are registered in the following sessions of this course

     ['2025.2', 'open']

and you have explicitly requested session '2025.2'.
your grades will be reported in that session

-------------
using course session ai4eng.v1.udea.r3::2025.2
success!! you are logged in
-------------


## Understand the ASHRAE Energy Prediction in Kaggle

Follow this link: [https://www.kaggle.com/c/ashrae-energy-prediction](https://www.kaggle.com/c/ashrae-energy-prediction), register for the task and download the data.

In particular, understand:

- The machine learning task they define (what they want to predict)
- How the data is provided
- The structure and relation between the files `train.csv`, `weather_train.csv` and `building_metadata.csv`

## Task 1: Build a training dataset

Execute the cell below and you will be given a **building_id** and a **date**. You will have to:

1. Build a table containing one row per hour and the following columns **only for that building_id and date**:
       'meter_reading', 'site_id', 'air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed', 'square_feet', 'year_built'
   Note that you will have to gather this information from the different csv sources.
   
1. use only measures with `meter`=0
1. Fill any missing values with zero.
1. Extract the column of the target variable and sum all values.
1. Sum all the values of the rest of the columns.
1. Fill in those values in the variables below.
1. Submit your answer.

Observe that, with this dataset, we could train a model to make predictions of the target variable, taking the rest of the variables as input.

For instance, for building number 900, on 2016-02-01:
- there are 24 records
- the sum of all values of the target variable is 4941 (cropping decimals)
- the sum of all values of the rest of the variables is 2705422 (cropping decimals)

In [4]:
from IPython.display import Image
from local.lib import labutils
import numpy as np
bid, date = labutils.biddate_for_student(student.user_id)
print ("id:", bid)
print ("date:", date)

id: 990
date: 2016-02-08


In [5]:
import pandas as pd
import os

os.environ['KAGGLE_CONFIG_DIR']="."
!kaggle competitions download -c ashrae-energy-prediction -f building_metadata.csv
!kaggle competitions download -c ashrae-energy-prediction -f weather_train.csv
!kaggle competitions download -c ashrae-energy-prediction -f train.csv

building_metadata.csv: Skipping, found more recently modified local copy (use --force to force download)
... resuming from 7450075 bytes (-6114514 bytes left) ...
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1157, in competition_download_cli
    self.competition_download_file(competition, file_name, path, force,
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1090, in competition_download_file
    self.download_file(response, outfile, kaggle.http_client(), quiet,
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 2147, in download_file
    response = http_client.call(
               ^^^^^

In [ ]:
!unzip train.csv
!unzip weather_train.csv

In [7]:
train=pd.read_csv("train.csv")
weather=pd.read_csv("weather_train.csv")
building=pd.read_csv("building_metadata.csv")
print('Dataset Train:')
train.head()


Dataset Train:


,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [8]:
train['timestamp'].info()
t = train[(train.building_id == bid) & (train.meter == 0)].fillna(0)
t['timestamp'] = pd.to_datetime(t['timestamp'])
t['timestamp'].info()
t.head()

<class 'pandas.core.series.Series'>
RangeIndex: 20216100 entries, 0 to 20216099
Series name: timestamp
Non-Null Count     Dtype 
--------------     ----- 
20216100 non-null  object
dtypes: object(1)
memory usage: 154.2+ MB
<class 'pandas.core.series.Series'>
Index: 8778 entries, 1352 to 20215137
Series name: timestamp
Non-Null Count  Dtype         
--------------  -----         
8778 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 137.2 KB


,building_id,meter,timestamp,meter_reading
1352,990,0,2016-01-01 00:00:00,0.0
3648,990,0,2016-01-01 01:00:00,49.0
5944,990,0,2016-01-01 02:00:00,48.0
8242,990,0,2016-01-01 03:00:00,50.0
10537,990,0,2016-01-01 04:00:00,48.0


In [9]:
print('Dataset Building:')
building.head()

Dataset Building:


,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [10]:
b = building[(building.building_id == bid)].fillna(0)
b

,site_id,building_id,primary_use,square_feet,year_built,floor_count
990,9,990,Office,82939,0.0,0.0


In [ ]:
print('Dataset Weather:')
weather.head()


In [11]:
w = weather[weather.site_id == b.site_id.values[0]].fillna(0)
w['timestamp'] = pd.to_datetime(w['timestamp'])
w.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
78828,9,2016-01-01 01:00:00,10.0,0.0,3.9,0.0,1027.7,20.0,3.1
78829,9,2016-01-01 02:00:00,10.0,0.0,3.9,0.0,1028.0,0.0,2.1
78830,9,2016-01-01 03:00:00,10.0,0.0,3.9,0.0,1028.9,30.0,3.6
78831,9,2016-01-01 04:00:00,10.0,0.0,3.9,0.0,1029.1,10.0,3.6
78832,9,2016-01-01 05:00:00,9.4,0.0,3.9,0.0,1029.5,10.0,4.6


In [12]:
print(f"Filas en b: {len(b)}")
print(f"Filas en t: {len(t)}")
print(f"Filas en w: {len(w)}")

Filas en b: 1
Filas en t: 8778
Filas en w: 8780


In [23]:
df = pd.DataFrame(columns=[
    'meter_reading', 'site_id', 'air_temperature', 'cloud_coverage',
    'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
    'wind_direction', 'wind_speed', 'square_feet', 'year_built'])
t_filtered = t[
    (t['meter_reading'] == t.meter_reading) &
    (t['meter'] == 0) &
    (t['timestamp'].dt.date == pd.to_datetime(date).date())].copy()

df = t_filtered.merge(
    b[['building_id', 'site_id', 'square_feet', 'year_built']],
    on='building_id',
    how='left')
df = df.merge(
    w[['site_id', 'timestamp', 'air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed']],
    on=['site_id', 'timestamp'],
    how='left')
df = df.fillna(0)
df = df.drop(['timestamp','meter','building_id'],axis=1)

print(f"\n{'='*50}")
print(f"Dataset final: {len(df)} filas × {len(df.columns)} columnas")
print(f"{'='*50}")
print(df.head())



Dataset final: 24 filas × 11 columnas
   meter_reading  site_id  square_feet  year_built  air_temperature  \
0           62.0        9        82939         0.0             16.7   
1           64.0        9        82939         0.0             15.0   
2           50.0        9        82939         0.0             13.9   
3           51.0        9        82939         0.0             11.7   
4           52.0        9        82939         0.0             10.6   

   cloud_coverage  dew_temperature  precip_depth_1_hr  sea_level_pressure  \
0             0.0              0.0                0.0              1020.1   
1             0.0              0.0                0.0              1020.4   
2             0.0              0.0                0.0              1020.9   
3             0.0              0.0                0.0              1021.3   
4             0.0              0.0                0.0              1021.1   

   wind_direction  wind_speed  
0             0.0         2.6  
1      

In [24]:

df.head()


,meter_reading,site_id,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,62.0,9,82939,0.0,16.7,0.0,0.0,0.0,1020.1,0.0,2.6
1,64.0,9,82939,0.0,15.0,0.0,0.0,0.0,1020.4,180.0,2.1
2,50.0,9,82939,0.0,13.9,0.0,0.0,0.0,1020.9,210.0,1.5
3,51.0,9,82939,0.0,11.7,0.0,0.0,0.0,1021.3,220.0,1.5
4,52.0,9,82939,0.0,10.6,0.0,0.0,0.0,1021.1,0.0,0.0


#### provide your answer here, drop the decimals

In [25]:

# target variable sum
sumY =int(df['meter_reading'].sum())

# input variables sum
sumX =int(df.sum().sum() - sumY)

sumX, sumY

(2020393, 2414)

**submit your answer**

In [26]:
student.submit_task(globals(), task_id="task_01");

## Task 2. Time series missing data fix

Observe how we can fill in missing data in a time series, by simply repeating the last seen value in the missing places.

In [ ]:
Image("local/imgs/timeseries-ffill.png")

Using the weather data from the ASHRAE Kaggle competition that you must have downloaded in the previous task you will have to:

1. Extract the time series for the variable `cloud_coverage` for **site_id=3** and for the **date assigned to you** (execute the cell below). This time series will have 24 values (including missing values).
1. Fill in the missing data by repeating the last seen value as in the example above
1. Report the time series as a Pyton list in the variable `fixed_ts` below

Again, use any tool of your choice to generate your result

In [ ]:
from local.lib import labutils
_, date = labutils.biddate_for_student(student.user_id)
print ("your date       ", date)

#### fill in your fixed time series

In [ ]:
# for instance
# fixed_ts = [6,6,6,4,4,5,6,7,7,7,6,6,6,5,4,3,2,2,1,1,1,2,3,4]
fixed_ts = [  ]

**submit your answer**

In [ ]:
student.submit_task(globals(), task_id="task_02");

## Task 3. Build a time series predictive dataset

You will now build a predictive dataset for a single time series. This dataset can later be used with predictive models. The machine learning task we will try to adress is the following:

> **Given three consecutive points of a time series, we want a model to predict the next one.**

Starting from a time series (a list of of $n$ numbers) you will have to build a dataset such as in the following example



In [ ]:

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ts = np.r_[44, 45, 42, 38, 47, 45, 46, 44, 47, 52, 53, 48, 46, 42, 42, 34, 30, 27, 24, 24]
plt.figure(figsize=(10,2))
plt.xticks(range(len(ts)), range(len(ts)))
plt.grid();
plt.plot(ts)
print (ts)


dataset:

    [[[44, 45, 42], 38],
     [[45, 42, 38], 47],
     [[42, 38, 47], 45],
     [[38, 47, 45], 46],
     [[47, 45, 46], 44],
     [[45, 46, 44], 47],
     [[46, 44, 47], 52],
     [[44, 47, 52], 53],
     [[47, 52, 53], 48],
     [[52, 53, 48], 46],
     [[53, 48, 46], 42],
     [[48, 46, 42], 42],
     [[46, 42, 42], 34],
     [[42, 42, 34], 30],
     [[42, 34, 30], 27],
     [[34, 30, 27], 24],
     [[30, 27, 24], 24]]

observe that each line in the dataset represents one input-output pair, like a sliding window over the time series.

We created two components:

- a matrix $X$ of dimensions `(len(ts)-n,3)` gathering all inputs, in the case above with dimensions `(17,3)`
- a vector $y$ of dimension `(len(ts)-n)` gathering each corresponding expected output, in the case above with dimensions `(17)`.

In this task, you will have to:

1. Take the time series you created in the previous task, where you fixed the missing values.
1. Build $X$, $y$ such as above with $n=3$. If your time series has 24 data points, then X dimensions will be (21,3) and y dimensions will be (21)

Again, use any tool of your choice to generate your result.

#### fill in your answer


In [ ]:
X = [ [  ,  ,  ],
      [  ,  ,  ],
      [  ,  ,  ]
       ....
    ]

y = [ , , , , ... ]

**submit your answer**

In [ ]:
student.submit_task(globals(), task_id="task_03");